In [1]:
%pip install python-dotenv langchain-openai langchain_community chromadb unstructured[html] langchain_hub

zsh:1: no matches found: unstructured[html]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import warnings
from unstructured.staging.base import dict_to_elements
from unstructured.partition.html import partition_html
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

_ = load_dotenv()

In [3]:
from unstructured.staging.base import dict_to_elements
from unstructured.partition.html import partition_html

Extract tables as text

In [4]:
file_path = "../docs/"

raw_html_elements = partition_html(
    filename=file_path + "nvidia_financial_results_q1_fiscal_2025.html",
    )

In [9]:
tables = [el for el in raw_html_elements if el.category == "Table"]

In [6]:
from io import StringIO 
from lxml import etree

table_html = tables[3].metadata.text_as_html

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
from IPython.core.display import HTML
HTML(table_html)

In [7]:
# loop through the tables and extract html
table_in_html = []
for table in tables:
    table_html = table.metadata.text_as_html
    table_in_html.append(table_html)
    print(table_html)

<table><tr><td>GAAP</td></tr><tr><td>($ in millions, except earnings per share)</td><td>Q1 FY25</td><td>Q4 FY24</td><td>Q1 FY24</td><td>Q/Q</td><td>Y/Y</td></tr><tr><td>Revenue</td><td>$26,044</td><td>$22,103</td><td>$7,192</td><td>Up 18%</td><td>Up 262%</td></tr><tr><td>Gross margin</td><td>78.4%</td><td>76.0%</td><td>64.6%</td><td>Up 2.4 pts</td><td>Up 13.8 pts</td></tr><tr><td>Operating expenses</td><td>$3,497</td><td>$3,176</td><td>$2,508</td><td>Up 10%</td><td>Up 39%</td></tr><tr><td>Operating income</td><td>$16,909</td><td>$13,615</td><td>$2,140</td><td>Up 24%</td><td>Up 690%</td></tr><tr><td>Net income</td><td>$14,881</td><td>$12,285</td><td>$2,043</td><td>Up 21%</td><td>Up 628%</td></tr><tr><td>Diluted earnings per share</td><td>$5.98</td><td>$4.93</td><td>$0.82</td><td>Up 21%</td><td>Up 629%</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td></tr></table>
<table><tr><td>Non-GAAP</td></tr><tr><td>($ in millions, except earnings per share)</td><td>Q1 FY25</td><td

Extract text from the html

In [8]:
file_path = "../docs/"

raw_html_elements = partition_html(
    filename=file_path + "nvidia_financial_results_q1_fiscal_2025.html",
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=file_path,
)

In [9]:
from pydantic import BaseModel
from typing import Any

class Element(BaseModel):
    type: str
    text: Any

categorized_elements = []
for element in raw_html_elements:
    if "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print("Text:", len(text_elements))

Text: 9


Save the table text to the Vector store

In [10]:
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

vectorstore = Chroma.from_texts(texts=table_in_html, 
                                    embedding=OpenAIEmbeddings())

Save text to the Vector store

In [11]:
# loop through the text and add to the vectorstore
vectorstore.aadd_texts(text_elements, embedding=OpenAIEmbeddings())

<coroutine object VectorStore.aadd_texts at 0x31f05a340>

Test the table data retreaval from the Vector store

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
retriever.invoke("Whats the basic net income per share for the three months ended April 28, 2024?")

[Document(page_content='<table><tr><td></td></tr><tr><td>NVIDIA CORPORATION</td></tr><tr><td>CONDENSED CONSOLIDATED STATEMENTS OF INCOME</td></tr><tr><td>(In millions, except per share data)</td></tr><tr><td>(Unaudited)</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td></td><td></td><td>Three Months Ended</td></tr><tr><td></td><td></td><td></td><td>April 28,</td><td></td><td>April 30,</td></tr><tr><td></td><td></td><td></td><td>2024</td><td></td><td>2023</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Revenue</td><td>$</td><td>26,044</td><td></td><td></td><td>$</td><td>7,192</td><td></td></tr><tr><td>Cost of revenue</td><td></td><td>5,638</td><td></td><td></td><td></td><td>2,544</td><td></td></tr><tr><td>Gross profit</td><td></td><td>20,406</td><td></td><td></td><td></td><td>4,648</td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></t

Test the text data retreaval

In [15]:
retriever.invoke("`whats the next industrial revolution?")

[Document(page_content='<table><tr><td>NVIDIA CORPORATION</td><td></td></tr><tr><td>CONDENSED CONSOLIDATED BALANCE SHEETS</td><td></td></tr><tr><td>(In millions)</td><td></td></tr><tr><td>(Unaudited)</td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td></td><td></td><td></td><td>April 28,</td><td></td><td>January 28,</td><td></td></tr><tr><td></td><td></td><td></td><td></td><td>2024</td><td></td><td>2024</td><td></td></tr><tr><td>ASSETS</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Current assets:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td>Cash, cash equivalents and marketable securities</td><td></td><td>$</td><td>31,438</td><td></td><td>$</td><td>25,984</td><td></td></tr><tr><td></td><td>Accounts receivable, net</td><td>

RAG Impl

In [16]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Test for table data

In [17]:
rag_chain.invoke("Whats the basic net income per share for the three months ended April 28, 2024?")

'The basic net income per share for the three months ended April 28, 2024, is $6.04.'

Test for text data

In [19]:
rag_chain.invoke("whats the next industrial revolution?")

'The next industrial revolution is often referred to as Industry 4.0. It involves the integration of digital technologies such as artificial intelligence, the Internet of Things (IoT), and advanced robotics into manufacturing and industrial processes. This revolution aims to create smart factories with increased automation, data exchange, and real-time monitoring.'